In [1]:
from ase_interface import ANIENS
from ase_interface import ensemblemolecule

import pyNeuroChem as pync
import pyaniasetools as pya
import hdnntools as hdt

import numpy as np
import  ase
import time
#from ase.build import molecule
#from ase.neb import NEB
#from ase.calculators.mopac import MOPAC
from ase.md.langevin import Langevin
from ase.md.verlet import VelocityVerlet
from ase.io.trajectory import Trajectory
from ase import units

from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

from ase.optimize.fire import FIRE as QuasiNewton

from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger

#from ase.neb import NEBtools
from ase.io import read, write
from ase.optimize import BFGS, LBFGS

import matplotlib
import matplotlib as mpl

import matplotlib.pyplot as plt

import seaborn as sns

van Der Waals correction will be unavailable. Please install ased3


In [2]:
dir = '/scratch/Research/gutzwiller_research/gutz_new_data/ML-GQMD-data-NEW/case1/train_models/data-U0.001/testset/'

# Molecule file
molfile = dir + 'sys0.xyz'

# Dynamics file
xyzfile = dir + 'mdcrd.xyz'

# Trajectory file
trajfile = dir + 'traj.dat'

# Optimized structure out:
optfile = dir + 'optmol.xyz'

ntdir = '/scratch/Research/gutzwiller_research/gutz_new_data/ML-GQMD-data-NEW/case1/train_models/data-U0.001/testsets/testset0'

T = 100.0 # Temperature
dt = 0.5
C = 0.5 # Optimization convergence
steps = 40000

ntdir = '/scratch/Research/gutzwiller_research/gutz_new_data/ML-GQMD-data-NEW/case1/train_models/data-U0.001/'
cns = ntdir + '../../params/rX-2.5R_32-2.0A_a8-8.params'
sae = ntdir + '../../params/sae.dat'
nnf = ntdir + 'train'
Nn = 3

#ntdir = '/nh/nest/u/jsmith/Research/gutzwiller_research/DarwinSets/GutzTrain/train_test_U2/U2_co3/'
#cns = ntdir + 'rX-6.0A_32-4.0A_a8-8.params'
#sae = ntdir + 'sae.dat'
#nnf = ntdir + 'train'
#Nn = 5

In [3]:
# Load molecule
mol = read(molfile)
L = 10.9753
mol.set_cell(([[L, 0, 0],
               [0, L, 0],
               [0, 0, L]]))

mol.set_pbc((True, True, True))

In [4]:
# Set NC
aens = ensemblemolecule(cns, sae, nnf, Nn, 1)

# Set ANI calculator
mol.set_calculator(ANIENS(aens,sdmx=20000000.0))

In [5]:
def erg_exp (Rmag):
    return 100.0*np.exp(-3.53*Rmag)

def frc_exp (Rmag,R):
    F = -3.53*erg_exp(Rmag)*(R/Rmag)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
    return F

mol.calc.set_pairwise(erg_exp, frc_exp)

In [6]:
# Optimize molecule
start_time = time.time()
dyn = QuasiNewton(mol)
dyn.run(fmax=C)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

# Save optimized mol
spc = mol.get_chemical_symbols()
pos = mol.get_positions(wrap=False).reshape(1,len(spc),3)

hdt.writexyzfile(optfile, pos, spc)

      Step     Time          Energy         fmax
FIRE:    0 08:16:23    -7356.890471      563.4587
FIRE:    1 08:16:23    -7832.635769      475.1397
FIRE:    2 08:16:23    -8237.435819      400.4996
FIRE:    3 08:16:24    -8577.416855      335.3246
FIRE:    4 08:16:24    -8851.244799      272.3658
FIRE:    5 08:16:24    -9048.194310      179.6813
FIRE:    6 08:16:24    -9176.643683      123.1064
FIRE:    7 08:16:24     2528.219929      692.9532
FIRE:    8 08:16:24     1190.374186      640.9157
FIRE:    9 08:16:24      -61.250044      592.0539
FIRE:   10 08:16:24    -1231.869651      547.5149
FIRE:   11 08:16:24    -2326.488793      506.2125
FIRE:   12 08:16:24    -3349.762951      467.8339
FIRE:   13 08:16:24    -4305.952889      432.3636
FIRE:   14 08:16:24    -5199.141509      400.1809
FIRE:   15 08:16:24    -6033.345602      370.3076
FIRE:   16 08:16:24    -6812.515162      342.1040
FIRE:   17 08:16:24    -7540.182129      317.1271
FIRE:   18 08:16:24    -8219.399701      312.3356
F

FIRE:  163 08:16:29   -12467.919774       69.3577
FIRE:  164 08:16:29   -12469.150863       72.5634
FIRE:  165 08:16:29   -12470.842646       65.7016
FIRE:  166 08:16:29   -12472.548873       33.6966
FIRE:  167 08:16:29   -12473.052830       44.3161
FIRE:  168 08:16:29   -12473.116066       42.1553
FIRE:  169 08:16:29   -12473.234600       37.9986
FIRE:  170 08:16:29   -12473.395097       32.1700
FIRE:  171 08:16:29   -12473.580323       26.9156
FIRE:  172 08:16:29   -12473.774151       25.3696
FIRE:  173 08:16:29   -12473.964354       23.7662
FIRE:  174 08:16:29   -12474.146052       22.2028
FIRE:  175 08:16:29   -12474.340081       20.3829
FIRE:  176 08:16:29   -12474.555980       21.4184
FIRE:  177 08:16:29   -12474.813433       24.8648
FIRE:  178 08:16:29   -12475.134134       25.8780
FIRE:  179 08:16:29   -12475.534933       23.5709
FIRE:  180 08:16:29   -12476.014335       17.2755
FIRE:  181 08:16:29   -12476.549002       11.1623
FIRE:  182 08:16:29   -12477.127352       13.4321


FIRE:  327 08:16:33   -12555.901396       15.6537
FIRE:  328 08:16:33   -12555.939742       16.3577
FIRE:  329 08:16:33   -12555.997907       17.3989
FIRE:  330 08:16:33   -12556.081531       18.8807
FIRE:  331 08:16:33   -12556.197055       20.9626
FIRE:  332 08:16:33   -12556.351580       23.8891
FIRE:  333 08:16:34   -12556.586417       28.5605
FIRE:  334 08:16:34   -12556.974461       36.3595
FIRE:  335 08:16:34   -12557.694918       49.9820
FIRE:  336 08:16:34   -12559.228455       73.7434
FIRE:  337 08:16:34   -12562.772976      102.4269
FIRE:  338 08:16:34   -12567.654420       27.0552
FIRE:  339 08:16:34   -12559.587760      243.0372
FIRE:  340 08:16:34   -12563.166340      186.0705
FIRE:  341 08:16:34   -12567.090869       80.9332
FIRE:  342 08:16:34   -12568.005911       36.1847
FIRE:  343 08:16:34   -12568.027977       34.7264
FIRE:  344 08:16:34   -12568.067747       31.8693
FIRE:  345 08:16:34   -12568.116312       27.7643
FIRE:  346 08:16:34   -12568.163728       22.6941


FIRE:  491 08:16:38   -12600.210383        6.5509
FIRE:  492 08:16:38   -12600.403742       11.2428
FIRE:  493 08:16:38   -12600.622379        9.3025
FIRE:  494 08:16:38   -12600.860789        7.0295
FIRE:  495 08:16:38   -12601.110842        8.2256
FIRE:  496 08:16:38   -12601.442385        7.4582
FIRE:  497 08:16:38   -12601.826907        9.6752
FIRE:  498 08:16:38   -12602.223082        7.3447
FIRE:  499 08:16:38   -12602.612235       10.4788
FIRE:  500 08:16:38   -12603.044801        9.5718
FIRE:  501 08:16:39   -12603.346132       13.2080
FIRE:  502 08:16:39   -12603.637199       27.7784
FIRE:  503 08:16:39   -12603.749440        9.0733
FIRE:  504 08:16:39   -12603.661509       26.2330
FIRE:  505 08:16:39   -12603.745742       19.9226
FIRE:  506 08:16:39   -12603.854865        9.1069
FIRE:  507 08:16:39   -12603.927155        6.7791
FIRE:  508 08:16:39   -12603.958120       12.8929
FIRE:  509 08:16:39   -12603.961744       12.2419
FIRE:  510 08:16:39   -12603.967722       10.9707


FIRE:  655 08:16:43   -12619.008821       16.0779
FIRE:  656 08:16:43   -12619.030264        7.3076
FIRE:  657 08:16:43   -12619.021649        3.4188
FIRE:  658 08:16:43   -12619.022414        3.2183
FIRE:  659 08:16:43   -12619.023689        2.8274
FIRE:  660 08:16:43   -12619.025879        2.2648
FIRE:  661 08:16:43   -12619.028202        2.1479
FIRE:  662 08:16:43   -12619.030849        2.1473
FIRE:  663 08:16:43   -12619.033484        2.1489
FIRE:  664 08:16:43   -12619.035629        2.1505
FIRE:  665 08:16:43   -12619.038431        2.1506
FIRE:  666 08:16:43   -12619.040808        2.1512
FIRE:  667 08:16:43   -12619.043694        2.1486
FIRE:  668 08:16:43   -12619.046481        2.1459
FIRE:  669 08:16:43   -12619.049843        2.1435
FIRE:  670 08:16:43   -12619.053702        2.1399
FIRE:  671 08:16:43   -12619.058229        2.1367
FIRE:  672 08:16:43   -12619.064786        2.1306
FIRE:  673 08:16:44   -12619.074824        2.1253
FIRE:  674 08:16:44   -12619.088088        2.1266


FIRE:  819 08:16:48   -12633.617948       10.2349
FIRE:  820 08:16:48   -12633.651145        9.6450
FIRE:  821 08:16:48   -12633.681503       12.5463
FIRE:  822 08:16:48   -12633.717178       15.0691
FIRE:  823 08:16:48   -12633.765548       15.4702
FIRE:  824 08:16:48   -12633.833094       13.1318
FIRE:  825 08:16:48   -12633.912281        8.8234
FIRE:  826 08:16:48   -12633.989611        7.1986
FIRE:  827 08:16:48   -12634.063803        9.0049
FIRE:  828 08:16:48   -12634.148312        8.2876
FIRE:  829 08:16:48   -12634.243055        5.1953
FIRE:  830 08:16:48   -12634.331776        7.0953
FIRE:  831 08:16:48   -12634.422891        7.2792
FIRE:  832 08:16:48   -12634.520429        4.9410
FIRE:  833 08:16:48   -12634.603071        5.6780
FIRE:  834 08:16:48   -12634.672668        4.6813
FIRE:  835 08:16:48   -12634.737498        8.7253
FIRE:  836 08:16:48   -12634.845350        4.1158
FIRE:  837 08:16:48   -12634.930051        7.7620
FIRE:  838 08:16:48   -12635.139697       26.3580


FIRE:  983 08:16:52   -12643.395633        0.7022
FIRE:  984 08:16:52   -12643.395415        0.7766
FIRE:  985 08:16:52   -12643.396054        1.1664
FIRE:  986 08:16:52   -12643.398049        0.8837
FIRE:  987 08:16:53   -12643.401321        0.7069
FIRE:  988 08:16:53   -12643.405011        1.0444
FIRE:  989 08:16:53   -12643.408379        0.9968
FIRE:  990 08:16:53   -12643.410773        0.7104
FIRE:  991 08:16:53   -12643.413394        0.9466
FIRE:  992 08:16:53   -12643.419635        0.7180
FIRE:  993 08:16:53   -12643.428007        1.0860
FIRE:  994 08:16:53   -12643.433537        0.7264
FIRE:  995 08:16:53   -12643.441548        0.7363
FIRE:  996 08:16:53   -12643.456774        0.9788
FIRE:  997 08:16:53   -12643.457138        2.1721
FIRE:  998 08:16:53   -12643.493372        5.2280
FIRE:  999 08:16:53   -12643.408644       12.8278
FIRE:  1000 08:16:53   -12643.502248        1.8213
FIRE:  1001 08:16:53   -12643.501339        1.3062
FIRE:  1002 08:16:53   -12643.498901        0.79

FIRE:  1144 08:16:57   -12644.199781        1.0620
FIRE:  1145 08:16:57   -12644.201139        0.9809
FIRE:  1146 08:16:57   -12644.201868        0.9740
FIRE:  1147 08:16:57   -12644.203207        0.9722
FIRE:  1148 08:16:57   -12644.207481        0.9536
FIRE:  1149 08:16:57   -12644.212733        0.9394
FIRE:  1150 08:16:57   -12644.216410        0.9189
FIRE:  1151 08:16:57   -12644.219723        1.0590
FIRE:  1152 08:16:57   -12644.228546        0.8571
FIRE:  1153 08:16:57   -12644.234468        0.8123
FIRE:  1154 08:16:57   -12644.243714        0.7481
FIRE:  1155 08:16:57   -12644.254789        0.6795
FIRE:  1156 08:16:57   -12644.260092        2.9721
FIRE:  1157 08:16:57   -12644.282801       11.3766
FIRE:  1158 08:16:57   -12644.275927        1.9919
FIRE:  1159 08:16:57   -12644.277662        1.4020
FIRE:  1160 08:16:57   -12644.280440        0.5956
FIRE:  1161 08:16:58   -12644.282967        0.6439
FIRE:  1162 08:16:58   -12644.285113        1.3147
FIRE:  1163 08:16:58   -12644.2

In [7]:
MaxwellBoltzmannDistribution(mol, 100.0 * units.kB)

# We want to run MD with constant energy using the VelocityVerlet algorithm.
dyn = VelocityVerlet(mol, 0.01 * units.fs)

In [ ]:
# Open MD output
mdcrd = open(xyzfile,'w')

# Open MD output
traj = open(trajfile,'w')

# Define the printer
def storeenergy(a=mol, d=dyn, b=mdcrd, t=traj):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)

    stddev =  a.calc.stddev

    t.write(str(d.get_number_of_steps()) + ' ' + str(ekin / (1.5 * units.kB)) + ' ' + str(epot) + ' ' + str(ekin) + ' ' + str(epot+ekin) + '\n')
    b.write(str(len(a)) + '\n' + str(ekin / (1.5 * units.kB)) + ' Step: ' + str(d.get_number_of_steps()) + '\n')
    c = a.get_positions(wrap=True)
    for j, i in zip(a, c):
        b.write(str('C') + ' ' + str(i[0]) + ' ' + str(i[1]) + ' ' + str(i[2]) + '\n')

    print('Step: %d Energy per atom: Epot = %.3f  Ekin = %.3f (T=%.3fK)  '
          'Etot = %.6f' ' StdDev = %.3f' % (d.get_number_of_steps(), epot, ekin, ekin / (1.5 * units.kB), epot + ekin, stddev))
dyn.attach(storeenergy, interval=25)

In [ ]:
start_time = time.time()
dyn.run(1000000)
print('[ANI Total time:', time.time() - start_time, 'seconds]')


Step: 25 Energy per atom: Epot = -63.219  Ekin = 0.009 (T=70.978K)  Etot = -63.210027 StdDev = 228.753
Step: 50 Energy per atom: Epot = -63.219  Ekin = 0.009 (T=68.880K)  Etot = -63.210158 StdDev = 228.741
Step: 75 Energy per atom: Epot = -63.217  Ekin = 0.007 (T=51.543K)  Etot = -63.210744 StdDev = 228.701
Step: 100 Energy per atom: Epot = -63.217  Ekin = 0.006 (T=48.868K)  Etot = -63.211087 StdDev = 228.676
Step: 125 Energy per atom: Epot = -63.219  Ekin = 0.008 (T=63.710K)  Etot = -63.210652 StdDev = 228.722
Step: 150 Energy per atom: Epot = -63.217  Ekin = 0.007 (T=54.852K)  Etot = -63.210013 StdDev = 228.793
Step: 175 Energy per atom: Epot = -63.217  Ekin = 0.007 (T=55.052K)  Etot = -63.210275 StdDev = 228.763
Step: 200 Energy per atom: Epot = -63.218  Ekin = 0.007 (T=56.885K)  Etot = -63.210940 StdDev = 228.689
Step: 225 Energy per atom: Epot = -63.219  Ekin = 0.008 (T=63.124K)  Etot = -63.210682 StdDev = 228.707
Step: 250 Energy per atom: Epot = -63.219  Ekin = 0.008 (T=65.480K)

Step: 1975 Energy per atom: Epot = -63.236  Ekin = 0.042 (T=325.963K)  Etot = -63.193777 StdDev = 228.947
Step: 2000 Energy per atom: Epot = -63.254  Ekin = 0.059 (T=455.141K)  Etot = -63.195048 StdDev = 228.821
Step: 2025 Energy per atom: Epot = -63.241  Ekin = 0.043 (T=332.720K)  Etot = -63.198084 StdDev = 228.608
Step: 2050 Energy per atom: Epot = -63.249  Ekin = 0.051 (T=392.177K)  Etot = -63.198128 StdDev = 228.562
Step: 2075 Energy per atom: Epot = -63.246  Ekin = 0.051 (T=395.996K)  Etot = -63.194835 StdDev = 228.764
Step: 2100 Energy per atom: Epot = -63.236  Ekin = 0.044 (T=337.598K)  Etot = -63.192820 StdDev = 228.867
Step: 2125 Energy per atom: Epot = -63.251  Ekin = 0.058 (T=446.878K)  Etot = -63.193335 StdDev = 228.836
Step: 2150 Energy per atom: Epot = -63.241  Ekin = 0.046 (T=358.455K)  Etot = -63.194697 StdDev = 228.742
Step: 2175 Energy per atom: Epot = -63.245  Ekin = 0.050 (T=384.176K)  Etot = -63.195427 StdDev = 228.579
Step: 2200 Energy per atom: Epot = -63.253  Ek

Step: 3925 Energy per atom: Epot = -63.365  Ekin = 0.203 (T=1568.758K)  Etot = -63.162123 StdDev = 228.592
Step: 3950 Energy per atom: Epot = -63.371  Ekin = 0.210 (T=1626.517K)  Etot = -63.160830 StdDev = 228.651
Step: 3975 Energy per atom: Epot = -63.321  Ekin = 0.161 (T=1243.456K)  Etot = -63.160254 StdDev = 228.790
Step: 4000 Energy per atom: Epot = -63.359  Ekin = 0.198 (T=1529.712K)  Etot = -63.161200 StdDev = 228.559
Step: 4025 Energy per atom: Epot = -63.341  Ekin = 0.181 (T=1403.004K)  Etot = -63.159190 StdDev = 228.715
Step: 4050 Energy per atom: Epot = -63.353  Ekin = 0.195 (T=1509.119K)  Etot = -63.157717 StdDev = 228.796
Step: 4075 Energy per atom: Epot = -63.375  Ekin = 0.215 (T=1666.457K)  Etot = -63.159250 StdDev = 228.439
Step: 4100 Energy per atom: Epot = -63.335  Ekin = 0.176 (T=1361.546K)  Etot = -63.158601 StdDev = 228.355
Step: 4125 Energy per atom: Epot = -63.399  Ekin = 0.245 (T=1895.571K)  Etot = -63.153740 StdDev = 228.432
Step: 4150 Energy per atom: Epot = -6

Step: 5850 Energy per atom: Epot = -63.496  Ekin = 0.414 (T=3206.555K)  Etot = -63.081404 StdDev = 228.336
Step: 5875 Energy per atom: Epot = -63.478  Ekin = 0.397 (T=3073.419K)  Etot = -63.080306 StdDev = 228.279
Step: 5900 Energy per atom: Epot = -63.493  Ekin = 0.418 (T=3233.185K)  Etot = -63.074789 StdDev = 228.591
Step: 5925 Energy per atom: Epot = -63.499  Ekin = 0.426 (T=3295.907K)  Etot = -63.072944 StdDev = 228.660
Step: 5950 Energy per atom: Epot = -63.470  Ekin = 0.398 (T=3079.174K)  Etot = -63.072086 StdDev = 228.769
Step: 5975 Energy per atom: Epot = -63.503  Ekin = 0.432 (T=3341.411K)  Etot = -63.070750 StdDev = 228.591
Step: 6000 Energy per atom: Epot = -63.507  Ekin = 0.436 (T=3371.706K)  Etot = -63.071467 StdDev = 228.334
Step: 6025 Energy per atom: Epot = -63.510  Ekin = 0.438 (T=3391.757K)  Etot = -63.071767 StdDev = 228.361
Step: 6050 Energy per atom: Epot = -63.524  Ekin = 0.452 (T=3494.588K)  Etot = -63.072119 StdDev = 228.317
Step: 6075 Energy per atom: Epot = -6

Step: 7775 Energy per atom: Epot = -63.583  Ekin = 0.544 (T=4204.710K)  Etot = -63.039470 StdDev = 229.373
Step: 7800 Energy per atom: Epot = -63.622  Ekin = 0.572 (T=4425.741K)  Etot = -63.049835 StdDev = 228.103
Step: 7825 Energy per atom: Epot = -63.601  Ekin = 0.558 (T=4313.699K)  Etot = -63.043035 StdDev = 228.823
Step: 7850 Energy per atom: Epot = -63.635  Ekin = 0.597 (T=4616.356K)  Etot = -63.037966 StdDev = 229.031
Step: 7875 Energy per atom: Epot = -63.625  Ekin = 0.585 (T=4524.819K)  Etot = -63.040048 StdDev = 228.737
Step: 7900 Energy per atom: Epot = -63.608  Ekin = 0.568 (T=4393.155K)  Etot = -63.039847 StdDev = 228.840
Step: 7925 Energy per atom: Epot = -63.580  Ekin = 0.535 (T=4139.280K)  Etot = -63.044853 StdDev = 228.423
Step: 7950 Energy per atom: Epot = -63.597  Ekin = 0.551 (T=4259.982K)  Etot = -63.046761 StdDev = 228.330
Step: 7975 Energy per atom: Epot = -63.584  Ekin = 0.543 (T=4200.725K)  Etot = -63.040772 StdDev = 229.155
Step: 8000 Energy per atom: Epot = -6

...

Step: 188300 Energy per atom: Epot = -63.950  Ekin = 0.973 (T=7528.588K)  Etot = -62.976949 StdDev = 227.513
Step: 188325 Energy per atom: Epot = -64.008  Ekin = 1.032 (T=7985.903K)  Etot = -62.975505 StdDev = 227.399
Step: 188350 Energy per atom: Epot = -64.005  Ekin = 1.032 (T=7980.523K)  Etot = -62.973407 StdDev = 227.547
Step: 188375 Energy per atom: Epot = -64.021  Ekin = 1.058 (T=8183.897K)  Etot = -62.963188 StdDev = 228.749
Step: 188400 Energy per atom: Epot = -63.982  Ekin = 0.992 (T=7672.431K)  Etot = -62.990716 StdDev = 226.419
Step: 188425 Energy per atom: Epot = -63.979  Ekin = 0.996 (T=7702.595K)  Etot = -62.983714 StdDev = 227.258
Step: 188450 Energy per atom: Epot = -64.045  Ekin = 1.074 (T=8307.868K)  Etot = -62.970640 StdDev = 228.385
Step: 188475 Energy per atom: Epot = -63.980  Ekin = 0.994 (T=7693.530K)  Etot = -62.985192 StdDev = 226.995
Step: 188500 Energy per atom: Epot = -64.018  Ekin = 1.042 (T=8058.733K)  Etot = -62.976214 StdDev = 227.912
Step: 188525 Energy

Step: 190200 Energy per atom: Epot = -64.015  Ekin = 1.057 (T=8176.972K)  Etot = -62.958381 StdDev = 227.282
Step: 190225 Energy per atom: Epot = -64.009  Ekin = 1.050 (T=8124.820K)  Etot = -62.958372 StdDev = 227.273
Step: 190250 Energy per atom: Epot = -64.007  Ekin = 1.046 (T=8095.329K)  Etot = -62.961034 StdDev = 226.982
Step: 190275 Energy per atom: Epot = -64.038  Ekin = 1.077 (T=8328.603K)  Etot = -62.961917 StdDev = 226.927
Step: 190300 Energy per atom: Epot = -64.037  Ekin = 1.084 (T=8385.429K)  Etot = -62.953584 StdDev = 227.674
Step: 190325 Energy per atom: Epot = -63.959  Ekin = 0.999 (T=7732.157K)  Etot = -62.959318 StdDev = 227.163
Step: 190350 Energy per atom: Epot = -64.043  Ekin = 1.079 (T=8349.714K)  Etot = -62.963261 StdDev = 226.750
Step: 190375 Energy per atom: Epot = -63.953  Ekin = 1.001 (T=7742.405K)  Etot = -62.951996 StdDev = 228.026
Step: 190400 Energy per atom: Epot = -64.022  Ekin = 1.063 (T=8223.477K)  Etot = -62.959061 StdDev = 226.822
Step: 190425 Energy

Step: 192100 Energy per atom: Epot = -64.013  Ekin = 1.069 (T=8273.925K)  Etot = -62.943072 StdDev = 227.820
Step: 192125 Energy per atom: Epot = -63.984  Ekin = 1.040 (T=8046.879K)  Etot = -62.943863 StdDev = 227.573
Step: 192150 Energy per atom: Epot = -64.056  Ekin = 1.102 (T=8527.693K)  Etot = -62.953227 StdDev = 226.901
Step: 192175 Energy per atom: Epot = -64.043  Ekin = 1.089 (T=8425.080K)  Etot = -62.954150 StdDev = 227.029
Step: 192200 Energy per atom: Epot = -63.991  Ekin = 1.033 (T=7989.210K)  Etot = -62.958178 StdDev = 226.942
Step: 192225 Energy per atom: Epot = -64.085  Ekin = 1.120 (T=8663.067K)  Etot = -62.964912 StdDev = 226.474
Step: 192250 Energy per atom: Epot = -64.085  Ekin = 1.135 (T=8779.960K)  Etot = -62.949916 StdDev = 227.887
Step: 192275 Energy per atom: Epot = -64.007  Ekin = 1.061 (T=8204.552K)  Etot = -62.946620 StdDev = 228.139
Step: 192300 Energy per atom: Epot = -64.002  Ekin = 1.039 (T=8041.583K)  Etot = -62.962119 StdDev = 226.878
Step: 192325 Energy

Step: 194000 Energy per atom: Epot = -64.105  Ekin = 1.153 (T=8921.676K)  Etot = -62.951964 StdDev = 227.143
Step: 194025 Energy per atom: Epot = -64.054  Ekin = 1.110 (T=8587.873K)  Etot = -62.943977 StdDev = 228.037
Step: 194050 Energy per atom: Epot = -64.066  Ekin = 1.110 (T=8587.281K)  Etot = -62.955605 StdDev = 226.944
Step: 194075 Energy per atom: Epot = -63.975  Ekin = 1.025 (T=7930.884K)  Etot = -62.950025 StdDev = 227.724
Step: 194100 Energy per atom: Epot = -64.069  Ekin = 1.115 (T=8622.620K)  Etot = -62.954473 StdDev = 226.881
Step: 194125 Energy per atom: Epot = -63.977  Ekin = 1.022 (T=7906.780K)  Etot = -62.954636 StdDev = 226.894
Step: 194150 Energy per atom: Epot = -64.036  Ekin = 1.092 (T=8446.810K)  Etot = -62.943798 StdDev = 227.707
Step: 194175 Energy per atom: Epot = -64.114  Ekin = 1.161 (T=8984.427K)  Etot = -62.952903 StdDev = 226.853
Step: 194200 Energy per atom: Epot = -64.026  Ekin = 1.076 (T=8325.840K)  Etot = -62.949549 StdDev = 227.169
Step: 194225 Energy

Step: 195900 Energy per atom: Epot = -64.005  Ekin = 1.065 (T=8237.859K)  Etot = -62.940006 StdDev = 228.215
Step: 195925 Energy per atom: Epot = -63.982  Ekin = 1.039 (T=8037.228K)  Etot = -62.943482 StdDev = 227.763
Step: 195950 Energy per atom: Epot = -64.003  Ekin = 1.050 (T=8125.684K)  Etot = -62.952575 StdDev = 227.007
Step: 195975 Energy per atom: Epot = -64.022  Ekin = 1.082 (T=8369.882K)  Etot = -62.939705 StdDev = 228.344
Step: 196000 Energy per atom: Epot = -64.017  Ekin = 1.070 (T=8279.296K)  Etot = -62.946926 StdDev = 227.774
Step: 196025 Energy per atom: Epot = -64.052  Ekin = 1.098 (T=8495.183K)  Etot = -62.953618 StdDev = 227.098
Step: 196050 Energy per atom: Epot = -63.967  Ekin = 1.019 (T=7883.853K)  Etot = -62.947547 StdDev = 227.937
Step: 196075 Energy per atom: Epot = -64.060  Ekin = 1.115 (T=8627.026K)  Etot = -62.945033 StdDev = 228.043
Step: 196100 Energy per atom: Epot = -63.970  Ekin = 1.024 (T=7918.720K)  Etot = -62.946847 StdDev = 228.256
Step: 196125 Energy